In [7]:
!pip install minio requests boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 24.3 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [24]:
# MinIO settings
INFERENCE_ENDPOINT = "https://granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
infer_url = f"{INFERENCE_ENDPOINT}/v1/completions"
LOG_FILE = "inference_logs.txt"

MINIO_ENDPOINT = "minio-api-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
MINIO_ACCESS_KEY = "minio"
MINIO_SECRET_KEY = "minio123"
MINIO_BUCKET = "logs"
MINIO_OBJECT_NAME = f"logs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"


In [25]:
import requests
import json
import boto3
from datetime import datetime
import os



In [26]:

def query_model(prompt, model="granite"):
    try:
        headers = {"Content-Type": "application/json"}
        payload = {
            "model": model,
            "prompt": prompt
        }

        response = requests.post(infer_url, headers=headers, json=payload)

        if response.status_code == 200:
            result = response.json()
            log_message = f"[{datetime.now()}] Prompt: {prompt}\nResponse: {json.dumps(result)}\n\n"
        else:
            log_message = f"[{datetime.now()}] Error {response.status_code}: {response.text}\n\n"
            result = {"error": response.status_code, "message": response.text}

        with open(LOG_FILE, "a") as log_file:
            log_file.write(log_message)

        return result

    except Exception as e:
        error_msg = f"[{datetime.now()}] Exception: {str(e)}\n\n"
        with open(LOG_FILE, "a") as log_file:
            log_file.write(error_msg)
        return {"exception": str(e)}


In [27]:


# Cell 5: Test the model
response = query_model("Hello, what can you do?")
print(response)


{'id': 'cmpl-062a1459-0489-426c-9303-f284e5ebc32f', 'object': 'text_completion', 'created': 1749630704, 'model': 'granite', 'choices': [{'index': 0, 'text': '\n\nI can provide information, answer questions, set reminders, manage tasks', 'logprobs': None, 'finish_reason': 'length', 'stop_reason': None, 'prompt_logprobs': None}], 'usage': {'prompt_tokens': 7, 'total_tokens': 23, 'completion_tokens': 16, 'prompt_tokens_details': None}}


In [28]:
response = query_model("Tell me a fun fact about space!")
print(response)

{'id': 'cmpl-6b1f54f3-563d-42c0-9278-5e628073ee48', 'object': 'text_completion', 'created': 1749630707, 'model': 'granite', 'choices': [{'index': 0, 'text': '\n\nCertainly! Did you know that a day on Venus', 'logprobs': None, 'finish_reason': 'length', 'stop_reason': None, 'prompt_logprobs': None}], 'usage': {'prompt_tokens': 8, 'total_tokens': 24, 'completion_tokens': 16, 'prompt_tokens_details': None}}


In [29]:
def upload_to_minio():
    try:
        s3_client = boto3.client(
            's3',
            endpoint_url=f"http://{MINIO_ENDPOINT}",
            aws_access_key_id=MINIO_ACCESS_KEY,
            aws_secret_access_key=MINIO_SECRET_KEY
        )

        # Check or create the bucket
        try:
            s3_client.head_bucket(Bucket=MINIO_BUCKET)
        except:
            s3_client.create_bucket(Bucket=MINIO_BUCKET)

        # Upload log file
        s3_client.upload_file(LOG_FILE, MINIO_BUCKET, MINIO_OBJECT_NAME)
        print(f"✅ Uploaded log to MinIO at: s3://{MINIO_BUCKET}/{MINIO_OBJECT_NAME}")

    except Exception as e:
        print(f"❌ Failed to upload log to MinIO: {e}")


In [30]:
upload_to_minio()


✅ Uploaded log to MinIO at: s3://logs/logs_20250611_083142.txt


In [31]:
def list_logs():
    s3_client = boto3.client(
        's3',
        endpoint_url=f"http://{MINIO_ENDPOINT}",
        aws_access_key_id=MINIO_ACCESS_KEY,
        aws_secret_access_key=MINIO_SECRET_KEY
    )

    objects = s3_client.list_objects_v2(Bucket=MINIO_BUCKET)
    for obj in objects.get('Contents', []):
        print(obj['Key'])


In [32]:
list_logs()


logs_20250611_083142.txt


In [34]:
def show_log_from_minio(object_name):
    try:
        s3_client = boto3.client(
            's3',
            endpoint_url=f"http://{MINIO_ENDPOINT}",
            aws_access_key_id=MINIO_ACCESS_KEY,
            aws_secret_access_key=MINIO_SECRET_KEY
        )

        # Download the object content into memory
        response = s3_client.get_object(Bucket=MINIO_BUCKET, Key=object_name)
        content = response['Body'].read().decode('utf-8')

        print(f"📄 Contents of {object_name}:\n")
        print(content)

    except Exception as e:
        print(f"❌ Failed to read log from MinIO: {e}")


In [36]:
show_log_from_minio("logs_20250611_083142.txt")


📄 Contents of logs_20250611_083142.txt:

[2025-06-11 08:19:30.123568] Error 404: {"detail":"Not Found"}

[2025-06-11 08:19:30.123674] Exception: cannot access local variable 'result' where it is not associated with a value

[2025-06-11 08:20:08.445011] Error 404: {"detail":"Not Found"}

[2025-06-11 08:20:08.445133] Exception: cannot access local variable 'result' where it is not associated with a value

[2025-06-11 08:20:53.726280] Error 404: {"detail":"Not Found"}

[2025-06-11 08:28:07.365958] Error 400: {"object":"error","message":"[{'type': 'missing', 'loc': ('body', 'model'), 'msg': 'Field required', 'input': {'inputs': 'Hello, what can you do?'}}, {'type': 'missing', 'loc': ('body', 'prompt'), 'msg': 'Field required', 'input': {'inputs': 'Hello, what can you do?'}}]","type":"BadRequestError","param":null,"code":400}

[2025-06-11 08:29:10.563146] Prompt: Hello, what can you do?
Response: {"id": "cmpl-e624140a-d7cd-4f54-8ef0-77ef2f7be44a", "object": "text_completion", "created": 174